In [26]:
%pylab qt

import os
import itertools
import cv2
import numpy as np
from glob import glob
from scipy.signal import argrelextrema

import helpers.features as fh
import helpers.display as dh
import helpers.geometry as gh
reload(fh)
reload(dh)
reload(gh)

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_mesh/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_registered_outline/'

def extract_landmark_points(outline_points):
    landmark_definitions = [
        {
            'x_min': -1,
            'x_max': 0.25,
            'y_min': -1.5,
            'y_max': -0.75,
            'method': 'min',
            'dim': 'y'
        },
        {
            'x_min': -0.5,
            'x_max': 0.5,
            'y_min': -1.5,
            'y_max': -0.75,
            'method': 'max',
            'dim': 'y'
        },
        {
            'x_min': 0.25,
            'x_max': 1,
            'y_min': -1.5,
            'y_max': -0.75,
            'method': 'min',
            'dim': 'y'
        },
        {
            'x_min': -1.5,
            'x_max': 0,
            'y_min': -0.3,
            'y_max': 0.3,
            'method': 'min',
            'dim': 'x'
        },
        {
            'x_min': -1,
            'x_max': -0.25,
            'y_min': 0.75,
            'y_max': 1.5,
            'method': 'max',
            'dim': 'y'
        },
        {
            'x_min': -0.5,
            'x_max': 0.5,
            'y_min': 0.75,
            'y_max': 1.5,
            'method': 'min',
            'dim': 'y'
        },
        {
            'x_min': 0.25,
            'x_max': 1,
            'y_min': 0.75,
            'y_max': 1.5,
            'method': 'max',
            'dim': 'y'
        }
    ]
    landmarks = []
    
    for definition in landmark_definitions:
        method = np.argmax if definition['method'] == 'max' else np.argmin
        method_extrema = np.greater if definition['method'] == 'max' else np.less
        dim = 0 if definition['dim'] == 'y' else 1
        sort_dim = 1 - dim
        
        x_min = definition['x_min']
        x_max = definition['x_max']
        y_min = definition['y_min']
        y_max = definition['y_max']
        selector = outline_points[:, 1] >= x_min
        selector = np.logical_and(selector, outline_points[:, 1] <= x_max)
        selector = np.logical_and(selector, outline_points[:, 0] >= y_min)
        selector = np.logical_and(selector, outline_points[:, 0] <= y_max)

        possible_points = outline_points[selector, :]
        sorted_points = possible_points[np.argsort(possible_points[:, sort_dim]), :]
        #print(sorted_points)
        
        local_extrema = argrelextrema(sorted_points[:, dim], method_extrema, order=10)[0]
        if (len(local_extrema) > 0):
            index_of_landmark = local_extrema[0]
        else:
            index_of_landmark = method(sorted_points[:, dim])
        
        #print(index_of_landmark)
        
        landmarks.append(sorted_points[index_of_landmark, :])
    return np.array(landmarks)

def register_outline(reference, points, no_iterations = 1):
    landmarks_reference = extract_landmark_points(reference)

    for i in range(no_iterations):
        landmarks = extract_landmark_points(points)
        #Compute the transformation between the current source
        #and destination cloudpoint
        R, t = gh.estimate_rigid_transform(landmarks, landmarks_reference)
        R = np.array(R)
        t = np.array(t)
        #Transform the previous source and update the
        #current source cloudpoint
        points = (np.dot(R, points.transpose())).transpose() + np.tile(t, (points.shape[0], 1))
    
    return points

def do_registration(files):
    loaded = []
    transposed_and_scaled = []
    registered = []
    
    for f in files:
        tri = np.load(f)
        points = tri['points']
        triangles = tri['triangles']
        
        label = os.path.basename(f)
        label = ''.join([i if ord(i) < 128 else ' ' for i in label])
        
        outline_points, outline_edges = gh.extract_outline(points, triangles)
        
        loaded.append({
            'label': label,
            'filename': f,
            'points': outline_points,
            'edges': outline_edges
        })
    
    for outline in loaded:
        points = outline['points']
        
        centroid = mean(points, axis=0)
        points = points - np.tile(centroid, (len(points), 1))
        
        scale = sqrt(np.sum(np.power(points, 2)) / len(points))
        points = np.divide(points, scale)
        points[: ,0] = -points[:, 0]
        points, edges = gh.normalize_outline(points, outline['edges'])
        
        transposed_and_scaled.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'points': points,
            'edges': edges
        })
    
    reference = max(transposed_and_scaled, key=lambda o: o['points'].shape[0])['points']
    for outline in transposed_and_scaled:
        points = outline['points']
        edges = outline['edges']
        
        if not np.array_equal(points, reference):
            points = register_outline(reference, points)
            points, edges = gh.normalize_outline(points, edges)
        registered.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'points': points,
            'edges': edges
        })
    
    return registered

Populating the interactive namespace from numpy and matplotlib
Current base path: C:\Users\Stefan\Dropbox\Masterarbeit


`%matplotlib` prevents importing * from pylab and numpy


In [27]:
registered = do_registration(glob(DATA_PATH + '*.npz'))

In [24]:
dh.outlines(plt, registered)

(<matplotlib.figure.Figure at 0x31049ef0>,
 <matplotlib.axes._subplots.AxesSubplot at 0x499e4828>)

In [25]:
outline = registered[14]
landmarks = extract_landmark_points(outline['points'])
fig, axes = dh.outline(plt, outline, show_direction=True)
axes.plot(landmarks[:, 1], landmarks[:, 0], 'bo')

In [28]:
for outline in registered:
    basename = os.path.basename(outline['filename'])
    destination = os.path.join(TO_PATH, basename)
    
    np.savez(destination, points=outline['points'], edges=outline['edges'])